# TRAIN HANGUL-RNN

In [1]:
# -*- coding: utf-8 -*-
# Import Packages
import numpy as np
import tensorflow as tf
import collections
import argparse
import time
import os
from six.moves import cPickle
from TextLoader import *
from Hangulpy3 import *

# LOAD DATASET WITH TEXTLOADER

In [2]:
data_dir    = "nine_dreams"
batch_size  = 50
seq_length  = 50
data_loader = TextLoader(data_dir, batch_size, seq_length)
# This makes "vocab.pkl" and "data.npy" in "nine_dreams"   
#  from "nine_dreams/input.txt" 

loading preprocessed files


# VOCAB AND CHARS

In [3]:
vocab_size = data_loader.vocab_size
vocab = data_loader.vocab
chars = data_loader.chars
print ( "type of 'data_loader.vocab' is %s, length is %d" 
       % (type(data_loader.vocab), len(data_loader.vocab)) )
print ( "type of 'data_loader.chars' is %s, length is %d" 
       % (type(data_loader.chars), len(data_loader.chars)) )

type of 'data_loader.vocab' is <class 'dict'>, length is 76
type of 'data_loader.chars' is <class 'tuple'>, length is 76


# VOCAB: DICTIONARY (CHAR->INDEX)

In [4]:
print (data_loader.vocab)

{'?': 41, 'ㅆ': 22, "'": 49, 'ㅑ': 34, 'ㅔ': 20, '_': 69, '7': 71, 'ㄳ': 60, 'ㅏ': 3, 'ㅠ': 37, 'ㅟ': 35, 'ㅋ': 42, '(': 45, 'ㅡ': 8, '-': 65, ' ': 1, 'ㅌ': 30, 'ᴥ': 0, 'ㅜ': 16, 'ㅗ': 9, 'ㅢ': 25, 'ㄵ': 50, 'ㅙ': 53, '5': 63, '0': 73, 'ㄾ': 70, 'ㅂ': 17, 'ㅎ': 14, 'ㅣ': 6, 'ㅊ': 23, '.': 24, 'ㅒ': 74, 'ㅝ': 36, 'ㅕ': 18, '"': 28, 'ㄹ': 5, 'ㅇ': 2, 'ㄷ': 13, 'ㄺ': 48, 'ㅀ': 51, ',': 27, '>': 75, '1': 58, '6': 59, 'ㅈ': 15, 'ㅐ': 21, 'ㅓ': 11, 'ㅃ': 47, 'ㅄ': 43, 'ㄻ': 55, 'ㅍ': 31, 'ㅖ': 39, 'ㅚ': 33, '!': 52, ':': 57, '2': 62, 'ㄴ': 4, '9': 64, 'ㅉ': 40, 'ㄿ': 56, '\x1a': 72, '4': 67, 'ㄼ': 54, 'ㅞ': 61, '\n': 19, '3': 66, 'ㄶ': 44, 'ㄲ': 29, 'ㄸ': 32, 'ㅁ': 12, '8': 68, 'ㅅ': 10, ')': 46, 'ㅛ': 38, 'ㄱ': 7, 'ㅘ': 26}


# CHARS: LIST (INDEX->CHAR)

In [5]:
print (data_loader.chars)
# USAGE
print (data_loader.chars[0])

('ᴥ', ' ', 'ㅇ', 'ㅏ', 'ㄴ', 'ㄹ', 'ㅣ', 'ㄱ', 'ㅡ', 'ㅗ', 'ㅅ', 'ㅓ', 'ㅁ', 'ㄷ', 'ㅎ', 'ㅈ', 'ㅜ', 'ㅂ', 'ㅕ', '\n', 'ㅔ', 'ㅐ', 'ㅆ', 'ㅊ', '.', 'ㅢ', 'ㅘ', ',', '"', 'ㄲ', 'ㅌ', 'ㅍ', 'ㄸ', 'ㅚ', 'ㅑ', 'ㅟ', 'ㅝ', 'ㅠ', 'ㅛ', 'ㅖ', 'ㅉ', '?', 'ㅋ', 'ㅄ', 'ㄶ', '(', ')', 'ㅃ', 'ㄺ', "'", 'ㄵ', 'ㅀ', '!', 'ㅙ', 'ㄼ', 'ㄻ', 'ㄿ', ':', '1', '6', 'ㄳ', 'ㅞ', '2', '5', '9', '-', '3', '4', '8', '_', 'ㄾ', '7', '\x1a', '0', 'ㅒ', '>')
ᴥ


# TRAINING BATCH (IMPORTANT!!)

In [6]:
x, y = data_loader.next_batch()
print ("Type of 'x' is %s. Shape is %s" % (type(x), x.shape,))
print ("x looks like \n%s" % (x))
print
print ("Type of 'y' is %s. Shape is %s" % (type(y), y.shape,))
print ("y looks like \n%s" % (y))

Type of 'x' is <class 'numpy.ndarray'>. Shape is (50, 50)
x looks like 
[[19 19  1 ..., 23  0  4]
 [ 1 19 42 ...,  6  0  2]
 [ 2  8  0 ..., 19  1  1]
 ..., 
 [ 1  7  8 ...,  2  6  0]
 [ 6  4  0 ..., 17  9  7]
 [ 1 19 17 ..., 17 16  5]]
Type of 'y' is <class 'numpy.ndarray'>. Shape is (50, 50)
y looks like 
[[19  1  1 ...,  0  4  3]
 [19 42 11 ...,  0  2 20]
 [ 8  0  5 ...,  1  1 10]
 ..., 
 [ 7  8  0 ...,  6  0  1]
 [ 4  0 12 ...,  9  7  0]
 [19 17 16 ..., 16  5  0]]


# DEFINE A MULTILAYER LSTM NETWORK

In [7]:
rnn_size   = 512
num_layers = 3
grad_clip  = 5. # <= GRADIENT CLIPPING (PRACTICALLY IMPORTANT)
vocab_size = data_loader.vocab_size

# SELECT RNN CELL (MULTI LAYER LSTM)
unitcell = tf.nn.rnn_cell.BasicLSTMCell(rnn_size)
cell = tf.nn.rnn_cell.MultiRNNCell([unitcell] * num_layers)

# Set paths to the graph
input_data = tf.placeholder(tf.int32, [batch_size, seq_length])
targets    = tf.placeholder(tf.int32, [batch_size, seq_length])
initial_state = cell.zero_state(batch_size, tf.float32)

# Set Network
with tf.variable_scope('rnnlm'):
    softmax_w = tf.get_variable("softmax_w", [rnn_size, vocab_size])
    softmax_b = tf.get_variable("softmax_b", [vocab_size])
    embedding = tf.get_variable("embedding", [vocab_size, rnn_size])
    inputs = tf.split(1, seq_length, tf.nn.embedding_lookup(
            embedding, input_data))
    inputs = [tf.squeeze(input_, [1]) for input_ in inputs]

# Define functions

In [8]:
# Output of RNN
outputs, last_state = tf.nn.seq2seq.rnn_decoder(inputs, initial_state
                        , cell, loop_function=None, scope='rnnlm')
output = tf.reshape(tf.concat(1, outputs), [-1, rnn_size])
logits = tf.nn.xw_plus_b(output, softmax_w, softmax_b)

# Next word probability
probs = tf.nn.softmax(logits)
print ("FUNCTIONS READY")

FUNCTIONS READY


# DEFINE LOSS FUNCTION 

In [9]:
loss = tf.nn.seq2seq.sequence_loss_by_example([logits], # Input
    [tf.reshape(targets, [-1])], # Target
    [tf.ones([batch_size * seq_length])], # Weight
    vocab_size)
print ("LOSS FUNCTION")

LOSS FUNCTION


# DEFINE COST FUNCTION 

In [10]:
cost = tf.reduce_sum(loss) / batch_size / seq_length

# GRADIENT CLIPPING ! 
lr = tf.Variable(0.0, trainable=False) # <= LEARNING RATE 
tvars = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip)
_optm = tf.train.AdamOptimizer(lr)
optm = _optm.apply_gradients(zip(grads, tvars))

final_state = last_state
print ("그래프 준비됨")

그래프 준비됨


# OPTIMIZE NETWORK WITH LR SCHEDULING

In [11]:
num_epochs    = 500
save_every    = 1000
learning_rate = 0.0002
decay_rate    = 0.97

save_dir = 'nine_dreams'

# 세션 설정
session_conf = tf.ConfigProto()
session_conf.gpu_options.allow_growth = True

sess = tf.Session(config=session_conf)
sess.run(tf.global_variables_initializer())
summary_writer = tf.train.SummaryWriter(save_dir
                    , graph=sess.graph)
saver = tf.train.Saver(tf.global_variables())
for e in range(num_epochs): # for all epochs

    # LEARNING RATE SCHEDULING 
    sess.run(tf.assign(lr, learning_rate * (decay_rate ** e)))

    data_loader.reset_batch_pointer()
    state = sess.run(initial_state)
    for b in range(data_loader.num_batches):
        start = time.time()
        x, y = data_loader.next_batch()
        feed = {input_data: x, targets: y, initial_state: state}
        # Train!
        train_loss, state, _ = sess.run([cost, final_state, optm], feed)
        end = time.time()
        # PRINT 
        if b % 100 == 0:
            print ("%d/%d (epoch: %d), loss: %.3f, time/batch: %.3f"  
                   % (e * data_loader.num_batches + b
                    , num_epochs * data_loader.num_batches
                    , e, train_loss, end - start))
        # SAVE MODEL
        if (e * data_loader.num_batches + b) % save_every == 0:
            checkpoint_path = os.path.join(save_dir, 'model.ckpt')
            saver.save(sess, checkpoint_path
                       , global_step = e * data_loader.num_batches + b)
            print("model saved to {}".format(checkpoint_path))

0/99500 (epoch: 0), loss: 4.862, time/batch: 1.022
model saved to nine_dreams/model.ckpt
100/99500 (epoch: 0), loss: 2.560, time/batch: 0.082
199/99500 (epoch: 1), loss: 2.107, time/batch: 0.090
299/99500 (epoch: 1), loss: 1.786, time/batch: 0.083
398/99500 (epoch: 2), loss: 1.879, time/batch: 0.083
498/99500 (epoch: 2), loss: 1.682, time/batch: 0.082
597/99500 (epoch: 3), loss: 1.768, time/batch: 0.090
697/99500 (epoch: 3), loss: 1.618, time/batch: 0.083
796/99500 (epoch: 4), loss: 1.695, time/batch: 0.090
896/99500 (epoch: 4), loss: 1.569, time/batch: 0.083
995/99500 (epoch: 5), loss: 1.638, time/batch: 0.089
model saved to nine_dreams/model.ckpt
1095/99500 (epoch: 5), loss: 1.523, time/batch: 0.083
1194/99500 (epoch: 6), loss: 1.589, time/batch: 0.083
1294/99500 (epoch: 6), loss: 1.480, time/batch: 0.082
1393/99500 (epoch: 7), loss: 1.547, time/batch: 0.088
1493/99500 (epoch: 7), loss: 1.443, time/batch: 0.083
1592/99500 (epoch: 8), loss: 1.512, time/batch: 0.090
1692/99500 (epoch: 